## Lending Club Case Study

### Setting up the base and dividing each section into the steps performed. 
#### We will perform the below task on the data set and provide the result
1. We will get the data from the file (As data is provided in csv file).
2. Look at the data set and perform the EDA on it.
3. Exclude the column which are not relevant nd mentione the column which are to be used.
4. Provide the visualization on the data to support the result.

## Step 1 - Loading the Lending Data and Understanding the data

In [64]:
#Loading all the necessary library for usage
#Imports - Starts
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib as mpl
import math as math
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization
import plotly.express as px
#Imports - End

#Load Data from the data set file. It requires the file to be present in the local system 

raw_dataset = pd.read_csv("/Users/sing73/upgrad/ACP/Data_LendingClubCaseStudy/loan.csv")

# Getting the information of the data frame
raw_dataset.info()
#Looking at 1st 10 rows of the data frame 
raw_dataset.head(10)

# Looking at all the records in the data frame
#raw_dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


/Users/sing73/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning:

Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.



,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
5,1075269,1311441,5000,5000,5000.0,36 months,7.90%,156.46,A,A4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
6,1069639,1304742,7000,7000,7000.0,60 months,15.96%,170.08,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
7,1072053,1288686,3000,3000,3000.0,36 months,18.64%,109.43,E,E1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
8,1071795,1306957,5600,5600,5600.0,60 months,21.28%,152.39,F,F2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
9,1071570,1306721,5375,5375,5350.0,60 months,12.69%,121.45,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [65]:
# Function will be defined below which we have to use during the data processing
# Function Def - Starts 

# - We need to define the employee service duration in number for usage
def employee_service_duration_to_number(emp_yr):
    f_no = emp_yr.split(" ")[0].strip()
    if('<' in emp_yr ):
        return 0
    if ('+' in f_no):
        return f_no.split("+")[0].strip()
    else:
        return f_no

# Function Def - End



## Step 2 - Data Cleaning Process 

 We have gone through the data in csv format and read the data column in Excel with the help of data Dictionary provided.
 1. There are 111 column and 39717 rows in the data set, EDA activity will be performed on the data set to understand the data more accurately.
 2. We have to look at the column which are importnat for us and provide some analytical value such as loan_amount, term, grade, dti etc.
    which are providing the financial value to infer result. This means we have to look only the customer data and not the behavioral data.
 3. We have also observed the data inconsistancies, such as '%', '<' , '+', 'year' as the fileds which are part of important fileds hence we need to 
    to make the value proper if they are Numbers or string to identify which helps accurately form the visualization. 
 4. We are looking to substitue the values in column which have null values with the mean / median so that which make the Numericle sense to the data provided.
 5. We can make use of Visualization such as graph, charts, pi-charts to look at the data behaviors and the outliers.

In [66]:

# Getting the list of the records for column that are relevant for us
filtered_raw_data = raw_dataset[['loan_amnt','term','int_rate','installment','grade','sub_grade','emp_length','home_ownership','annual_inc',
                                 'verification_status','loan_status','dti','purpose', 'acc_now_delinq','addr_state',
                                 'pub_rec_bankruptcies','funded_amnt','funded_amnt_inv','title','mths_since_last_delinq']]

# We need to get the data for the loan_status where loan_status is Charged Off / Fully Paid and Current we cannot determine whether it will be defaulter or not
loan_status_data_set = filtered_raw_data [(filtered_raw_data['loan_status'].isin(['Charged Off','Fully Paid']))]


#Remove the % and from int_rate and  months from term 
loan_status_data_set['int_rate'] = loan_status_data_set['int_rate'].astype(str).str.replace("%", "").astype(float)
loan_status_data_set['term'] = loan_status_data_set['term'].astype(str).str.replace("months", "").astype(int)

loan_status_data_set.isnull().sum()




loan_amnt                    0
term                         0
int_rate                     0
installment                  0
grade                        0
sub_grade                    0
emp_length                 228
home_ownership               0
annual_inc                   0
verification_status          0
loan_status                  0
dti                          0
purpose                      0
acc_now_delinq               0
addr_state                   0
pub_rec_bankruptcies       118
funded_amnt                  0
funded_amnt_inv              0
title                        2
mths_since_last_delinq    3485
dtype: int64

In [67]:
loan_status_data_set.loan_status.unique() # check if the loan status is only 'Fully Paid' and 'Charged Off'

array(['Charged Off'], dtype=object)

In [68]:
#Since we have obtained the loan_status data we can update the data of interest and employee length. Interest contain % sign as non numeric value 
# and employee length has string so convert in  numbers for usage

loan_status_data_set['emp_length'] = loan_status_data_set['emp_length'].astype(str).apply(lambda x: employee_service_duration_to_number(x))

# replacing all NaN with 0 to avoid any error hence we have computed emp_length as 0 where the value is not present
loan_status_data_set['emp_length'] = loan_status_data_set['emp_length'].str.replace("nan","0")

# Convert the emp_length as numeric value
loan_status_data_set['emp_length'] = pd.to_numeric(loan_status_data_set['emp_length'])

print("Mean Of Employee Duration - ",loan_status_data_set['emp_length'].mean())
print("Median Of Employee Duration - ",loan_status_data_set['emp_length'].median())

#Verify the emp_length if it is only numeric values
loan_status_data_set.emp_length.unique()

loan_status_data_set

Mean Of Employee Duration -  5.499599037690457
Median Of Employee Duration -  5.0


,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,dti,purpose,acc_now_delinq,addr_state,pub_rec_bankruptcies,funded_amnt,funded_amnt_inv,title,mths_since_last_delinq
1,2500,60,15.27,59.83,C,C4,NaN,RENT,30000.0,Source Verified,Charged Off,1.00,car,0,GA,0.0,2500,2500.0,bike,NaN
8,5600,60,21.28,152.39,F,F2,4.0,OWN,40000.0,Source Verified,Charged Off,5.55,small_business,0,CA,0.0,5600,5600.0,Expand Business & Buy Debt Portfolio,NaN
9,5375,60,12.69,121.45,B,B5,NaN,RENT,15000.0,Verified,Charged Off,18.08,other,0,TX,0.0,5375,5350.0,Building my credit history.,NaN
12,9000,36,13.49,305.38,C,C1,NaN,RENT,30000.0,Source Verified,Charged Off,10.08,debt_consolidation,0,VA,0.0,9000,9000.0,freedom,NaN
14,10000,36,10.65,325.74,B,B2,3.0,RENT,100000.0,Source Verified,Charged Off,7.06,other,0,CA,0.0,10000,10000.0,Other Loan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39667,2500,36,12.80,84.00,D,D4,NaN,MORTGAGE,32000.0,Not Verified,Charged Off,20.00,home_improvement,0,NE,NaN,2500,675.0,Need to put in new countertops.,0.0
39668,2500,36,9.64,80.26,B,B4,NaN,MORTGAGE,75000.0,Not Verified,Charged Off,20.00,home_improvement,0,NE,NaN,2500,825.0,re-model bathroom,0.0
39669,6500,36,15.01,225.37,F,F1,9.0,MORTGAGE,213000.0,Not Verified,Charged Off,6.45,small_business,0,GA,NaN,6500,225.0,Business Financing Aug 2007,0.0
39678,1000,36,10.59,32.55,C,C2,2.0,MORTGAGE,33990.0,Not Verified,Charged Off,14.47,wedding,0,VA,NaN,1000,950.0,Wedding site deposit,0.0


In [69]:
loan_status_data_set['title'] = loan_status_data_set['title'].fillna("NA") # Updating the Null Title with NA

# Updating the Null Title with 0 as these are non delinquent member
loan_status_data_set['mths_since_last_delinq'] = loan_status_data_set['mths_since_last_delinq'].fillna("0")
 
# Convert the mths_since_last_delinq as numeric to numeric 
loan_status_data_set['mths_since_last_delinq'] = pd.to_numeric(loan_status_data_set['mths_since_last_delinq'])

loan_status_data_set.isnull().sum()

loan_amnt                   0
term                        0
int_rate                    0
installment                 0
grade                       0
sub_grade                   0
emp_length                639
home_ownership              0
annual_inc                  0
verification_status         0
loan_status                 0
dti                         0
purpose                     0
acc_now_delinq              0
addr_state                  0
pub_rec_bankruptcies      118
funded_amnt                 0
funded_amnt_inv             0
title                       0
mths_since_last_delinq      0
dtype: int64

In [70]:
# pub_rec_bankruptcies is the another column taken and we can look at the data segregation in the box plot 
fig = px.box(loan_status_data_set, y="pub_rec_bankruptcies", title="Number of public record bankruptcies")
fig.update_layout()
fig.show()

# As per the Box plot of this we cannot get much evidence hence we can drop this cloumn or we can make the value as 0 

In [71]:
# Updating the Null Title with 0 as these are non delinquent member
loan_status_data_set['pub_rec_bankruptcies'] = loan_status_data_set['pub_rec_bankruptcies'].fillna("0")
 

In [72]:
#Data frame after the cleanup 
loan_status_data_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5627 entries, 1 to 39688
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   loan_amnt               5627 non-null   int64  
 1   term                    5627 non-null   int64  
 2   int_rate                5627 non-null   float64
 3   installment             5627 non-null   float64
 4   grade                   5627 non-null   object 
 5   sub_grade               5627 non-null   object 
 6   emp_length              4988 non-null   float64
 7   home_ownership          5627 non-null   object 
 8   annual_inc              5627 non-null   float64
 9   verification_status     5627 non-null   object 
 10  loan_status             5627 non-null   object 
 11  dti                     5627 non-null   float64
 12  purpose                 5627 non-null   object 
 13  acc_now_delinq          5627 non-null   int64  
 14  addr_state              5627 non-null  

## Step 3 - Performing the EDA

In [73]:
# With the help of Customer verification data we can say that Not Verified Customers are less than Verified (Source Verified + Verified) which means that 

#labels
lab = loan_status_data_set["verification_status"].value_counts().keys().tolist()
#values
val = loan_status_data_set["verification_status"].value_counts().values.tolist()

trace = go.Pie(labels = lab ,
               values = val ,
               marker = dict(colors =  [ 'royalblue' ,'lime', 'yellow'],
                             line = dict(color = "white",
                                         width =  1.3)
                            ),
               rotation = 90,
               hoverinfo = "label+value+text",
               hole = .5
              )
layout = go.Layout(dict(title = "Customer Verification Data",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                       )
                  )

data = [trace]
fig = go.Figure(data = data,layout = layout)
py.iplot(fig)

In [74]:
#Home Ownership data shows that the loan applied by people on rent are in significant number

#labels
lab = loan_status_data_set["home_ownership"].value_counts().keys().tolist()
#values
val = loan_status_data_set["home_ownership"].value_counts().values.tolist()

trace = go.Pie(labels = lab ,
               values = val ,
               marker = dict(colors =  [ 'royalblue' ,'lime', 'yellow', 'red', 'pink'],
                             line = dict(color = "white",
                                         width =  1.3)
                            ),
               rotation = 90,
               hoverinfo = "label+value+text",
               hole = .5
              )
layout = go.Layout(dict(title = "Home Ownership Data",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                       )
                  )

data = [trace]
fig = go.Figure(data = data,layout = layout)
py.iplot(fig)

#### We have to look at the outliers, if its present in some fileds such as 

In [75]:
#function  for histogram for Customer Grades who applies for loan
fig = px.histogram(loan_status_data_set, x="grade")
fig.show()

In [76]:
#function  for histogram for customer based on the term chosen
fig = px.histogram(loan_status_data_set, x="term")
fig.show()

In [77]:
#function  for histogram for employment duration
fig = px.histogram(loan_status_data_set, x="emp_length", labels="Employee Duration", text_auto=True, title="Employee Employment Duration")
fig.update_layout(bargap=0.5, xaxis_title_text='Employee Duration', yaxis_title_text='Count of Employee')
fig.show()

In [78]:
fig = px.box(loan_status_data_set, y="loan_amnt", title="Loan Amount Outliers")
fig.update_layout(yaxis_title_text='Loan Amount')
fig.show()

In [79]:
fig = px.box(loan_status_data_set, y="annual_inc", title="Annual Income")
fig.update_layout(xaxis_title_text='Annual Income')
fig.show()

## With the below graph we can see that there are outliers in the annual income hence we need to do the treatment for them.

In [80]:
q90 = loan_status_data_set["annual_inc"].quantile(0.90)
p90 = loan_status_data_set[loan_status_data_set['annual_inc'] <= q90]

p90.describe()

,loan_amnt,term,int_rate,installment,emp_length,annual_inc,dti,acc_now_delinq,funded_amnt,funded_amnt_inv,mths_since_last_delinq
count,5064.000000,5064.000000,5064.000000,5064.000000,4480.000000,5064.000000,5064.000000,5064.0,5064.000000,5064.000000,5064.000000
mean,11362.356833,46.080569,13.690022,315.752626,5.404464,52016.061919,14.177962,0.0,11064.509281,10242.113852,13.836690
std,7563.538513,11.846666,3.616840,200.823533,3.430785,21189.409004,6.597185,0.0,7274.346506,7157.725436,22.573898
min,900.000000,36.000000,5.420000,22.790000,0.000000,4080.000000,0.000000,0.0,900.000000,0.000000,0.000000
25%,5000.000000,36.000000,11.140000,163.935000,2.000000,36000.000000,9.257500,0.0,5000.000000,4876.807500,0.000000
50%,10000.000000,36.000000,13.490000,277.160000,5.000000,50000.000000,14.490000,0.0,10000.000000,8617.310202,0.000000
75%,15412.500000,60.000000,16.290000,419.897500,10.000000,66000.000000,19.540000,0.0,15000.000000,14725.000000,23.000000
max,35000.000000,60.000000,24.400000,1302.690000,10.000000,103000.000000,29.850000,0.0,35000.000000,35000.000000,107.000000
